<a href="https://colab.research.google.com/github/danielmancovidales/Aplicaciones-en-finanzas/blob/main/Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [235]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor

In [236]:
# Lectura de base de datos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [237]:
df= pd.read_csv('/content/drive/MyDrive/analitica3/Financiera/df_final2.csv')
df.drop(['Unnamed: 0'],axis=1, inplace=True)

In [238]:
df = df.dropna(subset=['ciudad'])


In [239]:
df.head(2)

,eventos,valor_pagado,ciudad,cancer,epoc,diabetes,hipertension,enf_cardiovascular,genero,edad,cantidad_enfermedades,clasifica_diagn
0,1,715463.04,1.0,0,0,0,0,0,1,27,0,2
1,1,691983.39,1.0,0,1,0,0,0,0,42,1,2


In [240]:
columnas = ['edad','epoc','diabetes','clasifica_diagn','ciudad','hipertension','cancer','enf_cardiovascular']

In [241]:
y = df['valor_pagado']
X = df[columnas]

In [242]:
# Si los valores de y son continuos, convertirlos a enteros
if y.dtype != 'int':
    y = y.round().astype(int)

In [243]:
#Comprobar tamaño de variables
print("Tamaño de X :", X.shape)
print("Tamaño de y :", y.shape)

Tamaño de X : (26285, 8)
Tamaño de y : (26285,)


In [244]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [245]:
# Verificar las dimensiones después de la división
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)
print("Dimensiones de y_train:", y_train.shape)
print("Dimensiones de y_test:", y_test.shape)

Dimensiones de X_train: (21028, 8)
Dimensiones de X_test: (5257, 8)
Dimensiones de y_train: (21028,)
Dimensiones de y_test: (5257,)


In [246]:
scaler = StandardScaler()
# Ajustar el escalador solo en el conjunto de entrenamiento
X_train = scaler.fit_transform(X_train)
# Transformar el conjunto de prueba
X_test = scaler.transform(X_test)

In [247]:
# Verificar las dimensiones después del preprocesamiento
print("Dimensiones de X_train después del escalado:", X_train.shape)
print("Dimensiones de X_test después del escalado:", X_test.shape)

Dimensiones de X_train después del escalado: (21028, 8)
Dimensiones de X_test después del escalado: (5257, 8)


##Regresión Lineal

In [249]:
# Ajustar el modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir y evaluar el modelo
y_pred = model.predict(X_test)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
# Imprimir las métricas de evaluación
print("MSE:", mean_squared_error(y_test, y_pred))
print('R2:',r2_score(y_test, y_pred))
print("RMSE:", rmse)
print("MAE:", mae)

MSE: 5914485824411.961
R2: 0.035348596356535866
RMSE: 2433292.9189685117
MAE: 1747132.0037556915


##Regresión Ridge y Lasso

In [250]:
# Ridge Regression
ridge = Ridge(alpha=1.0)
ridge_scores = cross_val_score(ridge, X, y, cv=5, scoring='neg_mean_squared_error')
print("Ridge MSE:", -ridge_scores.mean())

# Lasso Regression
lasso = Lasso(alpha=0.1)
lasso_scores = cross_val_score(lasso, X, y, cv=5, scoring='neg_mean_squared_error')
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
print("Lasso MSE:", -lasso_scores.mean())
print('R2:',r2_score(y_test, y_pred))
# Imprimir las métricas de evaluación
print("RMSE:", rmse)
print("MAE:", mae)

Ridge MSE: 5857336061336.568
Lasso MSE: 5857340699454.616
R2: 0.035348596356535866
RMSE: 2433292.9189685117
MAE: 1747132.0037556915


##Random Forest

In [231]:
# Crear el modelo de regresión con Random Forest
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

# Entrenar el modelo
rf_regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = rf_regressor.predict(X_test)

# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir las métricas de evaluación
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R-squared:", r2)

MSE: 5312521786734.141
RMSE: 2304890.840524588
MAE: 1562733.0373864206
R-squared: 0.13352880527546263


##XGBoost

In [232]:
model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE:", mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)
print("MAE:", mae)
print("R-squared:", r2)

MSE: 5305282655550.235
RMSE: 2304890.840524588
MAE: 1560600.5554112373
R-squared: 0.13470950606079368


##Decisión tree Classifier

In [233]:
# Crear el modelo de regresión con Decision Tree Regressor
dt_regressor = DecisionTreeRegressor()

# Entrenar el modelo
dt_regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = dt_regressor.predict(X_test)

# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir las métricas de evaluación
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R-squared:", r2)

MSE: 6028680218050.424
RMSE: 2455337.0884769415
MAE: 1613284.9545046464
R-squared: 0.016723514585795463


##SGD Regressor

In [234]:
# Crear el modelo de regresión con SGDRegressor
sgd_regressor = SGDRegressor()

# Entrenar el modelo
sgd_regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = sgd_regressor.predict(X_test)

# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir las métricas de evaluación
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R-squared:", r2)


MSE: 5920914429502.3
RMSE: 2433292.9189685117
MAE: 1748598.7363645518
R-squared: 0.0343000922079133


# **Despliegue**

In [209]:
## se guarda el modelo random forest ya que entre todos los modelos aplicados es el mejor, con un R-squared: 0.13352826987170763

import pickle
nombreArchivo = 'modelo_final.sav'
pickle.dump(rf_regressor, open(nombreArchivo, 'wb'))

In [210]:
modeloCargado = pickle.load(open(nombreArchivo, 'rb'))
result = modeloCargado.score(X_test, y_test)


In [211]:
import joblib
joblib.dump(rf_regressor, "rf_model.pkl")

['rf_model.pkl']

In [212]:
model = joblib.load('rf_model.pkl')
y_predict = model.predict(X_test)
print(y_predict)

[1863268.17206501 3687481.71159333 3869260.00850048 ... 1394678.15539292
 1903992.99194177 1394678.15539292]


In [213]:
X.columns

Index(['edad', 'epoc', 'diabetes', 'clasifica_diagn', 'ciudad', 'hipertension',
       'cancer', 'enf_cardiovascular'],
      dtype='object')

In [214]:
data_pred={
    'edad': 40,
    'epoc': 1,
    'diabetes':0,
    'clasifica_diagn':3,
    'ciudad': 2,
    'hipertension':0,
    'cancer':0,
    'enf_cardiovascular': 1

}

In [215]:
data = np.array(list(data_pred.values())).reshape(1, -1)
y_predict = model.predict(data)
print('El costo del seguro es: ',y_predict[0] )

El costo del seguro es:  2871310.323932727
